# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**: Transformation and actions

**Fecha**: 21/02/2024

**Nombre del Estudiante**:
    - Luis Acosta
    - Samantha Quintero
    - Benjamin Vergara

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://be6296989c4d:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 14:11:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import importlib
import gatubelxs.spark_utils

importlib.reload(gatubelxs.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = gatubelxs.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

Transformations

In [21]:
from pyspark.sql.functions import desc, sum

In [ ]:
grouped_df = tiktok_df.groupBy("genre").avg("duration")

In [22]:
# Top 5 most energetic songs
sorted_df = tiktok_df.sort(desc("energy"))

In [23]:
# Total duration of songs in playlists
playlist_df = tiktok_df.groupBy("playlist_id").sum("duration_mins")

In [ ]:
grouped_df.show()

Actions

In [5]:
row_count=psongs_df.count()
row_count

1023

In [24]:
sorted_df.show(n=5)
playlist_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------------------+-------------------+--------+------------------+------------------+--------------------+--------------------+-----------------+----------+
|            track_id|          track_name|           artist_id|         artist_name|            album_id|duration|       release_date|popularity|      danceability|            energy|key|          loudness|mode|speechiness|        acousticness|   instrumentalness|liveness|           valence|             tempo|         playlist_id|       playlist_name|    duration_mins|     genre|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------

+--------------------+------------------+
|         playlist_id|sum(duration_mins)|
+--------------------+------------------+
|5IZc3KIVFhjzJ0L2k...| 7.474666666666667|
|08ia51KbTcfs4QVT5...|            4.1485|
|7xVLFuuYdAvcTfcP3...| 9.456433333333333|
|2RBILNmyq8p4fqVWO...| 2.162933333333333|
|6GdDjthxbTGBV9rl2...|3.3209166666666667|
|7krYEnB1OI1RbnJBa...|2.0957666666666666|
|1FgPyHX7HruKDL4Tx...|            2.4448|
|62RtxFf9epYNWOUHJ...|2.6694333333333335|
|5ow0sNF1zSqp71Ix5...|2.7334833333333335|
|0LlJbV4lyzJYE14YC...|10.709133333333334|
|6NFKf8vBApSvtzkap...|3.7074333333333334|
|5P8lyudWE7HQxb4lu...| 4.250666666666667|
|2BgEsaKNfHUdlh97K...| 3.116433333333333|
|7F9vK8hNFMml4GtHs...| 3.173783333333333|
|4vVTI94F9uJ8lHNDW...|3.3657666666666666|
|2uULRpRtKhCdojXwo...|               2.2|
|1tRlGMHsf21FDo6pj...|           1.79005|
|215fAfwkWtlj30ofd...|2.3214166666666665|
|3bidbhpOYeV4knp8A...|            4.3057|
|0YFocHKmrMme7Isel...| 4.810383333333333|
+--------------------+------------